In [1]:
import pandas as pd
import numpy as np
import sys
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col, mean, min, max
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GBTRegressor, RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

%load_ext autotime

In [2]:
!pwd

/home/ubuntu/Regression_4th_Iteration
time: 109 ms


In [3]:
#sc = SQLContext()
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
       .map(lambda line: line.split(','))
   #didn’t work with take(1). believe returns
   #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

#here’s the new vectorizer function:

def vectorizeData(data):
        return data.rdd.map(lambda r: [r[0], r[1], r[2], r[3], Vectors.dense(r[4:-1]),float(r[-1])])\
            .toDF(['route cluster', 'performance cluster', 'userid', 'workoutid', 'features', 'label'])

#load cluster csv
pandas_df = pd.read_csv('/home/ubuntu/endomondo_clusters.csv')
df = sqlContext.createDataFrame(pandas_df)

time: 35.1 s


In [4]:
pandas_df.columns

Index([u'Unnamed: 0', u'workoutid', u'route_prediction', u'perf_prediction',
       u'userid', u'diff_altitude', u'geo_distance', u'heart_rate_avg',
       u'speed_avg', u'elapsed_time', u'user_avg_speed', u'user_avg_dist'],
      dtype='object')

time: 3.71 ms


In [5]:
df.show(5)

+----------+---------+----------------+---------------+--------+-------------+--------------------+--------------+-------------+------------+--------------+--------------------+
|Unnamed: 0|workoutid|route_prediction|perf_prediction|  userid|diff_altitude|        geo_distance|heart_rate_avg|    speed_avg|elapsed_time|user_avg_speed|       user_avg_dist|
+----------+---------+----------------+---------------+--------+-------------+--------------------+--------------+-------------+------------+--------------+--------------------+
|         0|595437630|               1|              2| 7967148|660.200012207|0.037654705345600004|           0.0|          0.0|       476.0|           0.0|0.037654705345600004|
|         1|601204749|               1|              2|11314348|        848.0|      0.520372331142|           0.0|          0.0|     12204.0|           0.0| 0.38692112266999995|
|         2|561312883|               1|              0|11314348|594.400024414| 0.25346991419799997| 138.860244

In [6]:
df.schema

StructType(List(StructField(Unnamed: 0,LongType,true),StructField(workoutid,LongType,true),StructField(route_prediction,LongType,true),StructField(perf_prediction,LongType,true),StructField(userid,LongType,true),StructField(diff_altitude,DoubleType,true),StructField(geo_distance,DoubleType,true),StructField(heart_rate_avg,DoubleType,true),StructField(speed_avg,DoubleType,true),StructField(elapsed_time,DoubleType,true),StructField(user_avg_speed,DoubleType,true),StructField(user_avg_dist,DoubleType,true)))

time: 2.45 ms


In [7]:
select_columns = ['route_prediction', 'perf_prediction','userid', 'workoutid', 'geo_distance', 'diff_altitude', \
                'speed_avg', 'heart_rate_avg','user_avg_dist', 'user_avg_speed', 'elapsed_time']

reg_df = vectorizeData(df.select(select_columns))

time: 1.15 s


In [8]:
route_clusters = reg_df.select('route cluster').distinct().collect()
perf_clusters = reg_df.select('performance cluster').distinct().collect()
route_cluster_numbers = [int(route_clusters[i][0]) for i in range(len(route_clusters))]
perf_cluster_numbers = [int(perf_clusters[i][0]) for i in range(len(perf_clusters))]

time: 4.95 s


In [9]:
print "Cluster Summaries"
for n in perf_cluster_numbers:
    for i in route_cluster_numbers:
        temp_df = reg_df[(reg_df['route cluster'] == i) & (reg_df['performance cluster'] == n)]
        print 'Route%d_Perf%d:' % (i, n)
        temp_df.describe('label').show()

Cluster Summaries
Route0_Perf0:
+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|            103009|
|   mean|2098.6224698812725|
| stddev|1559.3188434959077|
|    min|               0.0|
|    max|           15453.0|
+-------+------------------+

Route1_Perf0:
+-------+-----------------+
|summary|            label|
+-------+-----------------+
|  count|              517|
|   mean| 9742.01160541586|
| stddev|6157.416510295563|
|    min|            198.0|
|    max|          30839.0|
+-------+-----------------+

Route3_Perf0:
+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|             41915|
|   mean|  4537.61619945127|
| stddev|2406.2365123232585|
|    min|               8.0|
|    max|           14334.0|
+-------+------------------+

Route2_Perf0:
+-------+-----------------+
|summary|            label|
+-------+-----------------+
|  count|             9136|
|   mean|4239.021891418564|
| stddev|

In [ ]:
#model_dictionary
model_dict = {}
model_dict['lr'] = {'model': LinearRegression(featuresCol="features", labelCol="label"),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxIter, [5, 10, 100]) \
                    .addGrid(temp_lr.regParam, [0, 0.1, 0.01]) \
                    .build()"}

model_dict['dt'] = {'model': DecisionTreeRegressor(featuresCol="features", labelCol="label",maxMemoryInMB=1028),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxDepth, [3, 5]) \
                    .addGrid(temp_lr.minInfoGain, [0, 0.1, 1]) \
                    .build()"}

model_dict['gbt'] = {'model': GBTRegressor(featuresCol="features", labelCol="label", maxMemoryInMB=2056),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxDepth, [3, 5]) \
                    .addGrid(temp_lr.maxIter, [10,20,40]) \
                    .build()"}

model_dict['rfr'] = {'model': RandomForestRegressor(featuresCol="features", labelCol="label", maxMemoryInMB = 2056),\
                    'param': "ParamGridBuilder() \
                    .addGrid(temp_lr.maxDepth, [3, 5]) \
                    .addGrid(temp_lr.numTrees, [10,20,40]) \
                    .build()"}

time: 63.8 ms


In [ ]:
for m in model_dict.keys():
    for n in perf_cluster_numbers:
        for i in route_cluster_numbers:
            temp_df = reg_df[(reg_df['route cluster'] == i) & (reg_df['performance cluster'] == n)]
            #temp_df1 = reg_df[reg_df['route cluster']==i]
            #temp_df = temp_df1[reg_df['performance cluster']==n]
            temp_df_cv = temp_df

            if temp_df.count() < 5:
                print "Cluster pair dropped: Route%d_Perf%d_%s" % (i,n,m)
                pass

            else:
                temp_lr = model_dict[m]['model']
                temp_lrModel = temp_lr.fit(temp_df['label','features'])
                temp_df = temp_lrModel.transform(temp_df)

                paramGrid = eval(model_dict[m]['param'])
                #paramGrid = ParamGridBuilder() \
                #.addGrid(temp_lr.maxDepth, [3, 5]) \
                #.addGrid(temp_lr.numTrees, [10,20,40]) \
                #.build()


                evaluator = RegressionEvaluator(
                    labelCol="label", predictionCol="prediction", metricName="mae")

                crossval = CrossValidator(estimator=temp_lr,
                                          estimatorParamMaps=paramGrid,
                                          evaluator=evaluator,
                                          numFolds=10)  # use 3+ folds in practice

                # Run cross-validation, and choose the best set of parameters.
                cvModel = crossval.fit(temp_df_cv)

                pred = cvModel.transform(temp_df_cv)


                #print("For " + str(type(temp_lr)) + " and cluster " + str(i))
                #print(cvModel.explainParams())

                #Print the coefficients and intercept for linear regression
                #print("Coefficients: " + str(temp_lrModel.coefficients))
                #print("Intercept: " + str(temp_lrModel.intercept))


                
                mae = evaluator.evaluate(pred)
                rmse = evaluator.evaluate(pred,{evaluator.metricName: "rmse"})
                r2 = evaluator.evaluate(pred, {evaluator.metricName: "r2"})

                model_dict[m]['Route%d_Perf%d_%s' % (i,n,m)]={}
                model_dict[m]['Route%d_Perf%d_%s' % (i,n,m)]['mae'] = mae
                model_dict[m]['Route%d_Perf%d_%s' % (i,n,m)]['rmse'] = rmse
                model_dict[m]['Route%d_Perf%d_%s' % (i,n,m)]['r2'] = r2
                

                
                print("(Route, Perf, Model): " + str((i,n,m)) +": " + str(rmse))
                print(cvModel.bestModel)
                #print(cvModel.bestModel.weights)
                #print(cvModel.bestModel.coefficients)
                basePath = "/home/ubuntu/Regression_4th_Iteration/Models/"
                cvModel.bestModel.save(basePath +"Route%d_Perf%d_%s" % (i,n,m))
                
                model_dict[m]['Route%d_Perf%d_%s' % (i,n,m)]["path"] = basePath +"Route%d_Perf%d_%s" % (n,i,m)
 

(Route, Perf, Model): (0, 0, 'rfr'): 960.56020737
RandomForestRegressionModel (uid=rfr_421be768b91e) with 20 trees
(Route, Perf, Model): (2, 0, 'lr'): 1177.94838983
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (4, 0, 'lr'): 2790.57792829
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (0, 1, 'lr'): 1060.52086606
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (1, 1, 'lr'): 5477.98221579
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (3, 1, 'lr'): 989.361385875
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (2, 1, 'lr'): 1465.48737067
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (4, 1, 'lr'): 2469.00265376
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (0, 2, 'lr'): 803.561676308
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (1, 2, 'lr'): 3135.22991931
LinearRegression_4f2c8fa7300214f1210b
(Route, Perf, Model): (3, 2, 'lr'): 1369.01025686
LinearRegression_4f2c8fa7300214f1210b
(Rout

In [15]:
model_dict

{'dt': {'Route0_Perf0_dt': {'mae': 526.2860448281214,
   'path': '/home/ubuntu/Regression_4th_Iteration/Models/Route0_Perf0_dt',
   'r2': 0.653202242038257,
   'rmse': 918.2711915465923},
  'Route0_Perf1_dt': {'mae': 598.5095908286135,
   'path': '/home/ubuntu/Regression_4th_Iteration/Models/Route1_Perf0_dt',
   'r2': 0.6601933338824966,
   'rmse': 936.6953747407589},
  'Route0_Perf2_dt': {'mae': 362.2477960766258,
   'path': '/home/ubuntu/Regression_4th_Iteration/Models/Route2_Perf0_dt',
   'r2': 0.8631485301130457,
   'rmse': 575.927891213726},
  'Route1_Perf0_dt': {'mae': 2281.4506561327416,
   'path': '/home/ubuntu/Regression_4th_Iteration/Models/Route0_Perf1_dt',
   'r2': 0.7331699692337391,
   'rmse': 3177.572470732732},
  'Route1_Perf1_dt': {'mae': 4010.716879502928,
   'path': '/home/ubuntu/Regression_4th_Iteration/Models/Route1_Perf1_dt',
   'r2': 0.5268307056311299,
   'rmse': 5133.906138575599},
  'Route1_Perf2_dt': {'mae': 1531.5838930965754,
   'path': '/home/ubuntu/Regres

time: 13.7 ms


In [16]:
!pwd

/home/ubuntu/Regression_4th_Iteration
time: 124 ms


In [19]:
model_dict.keys()

['rfr', 'dt', 'lr', 'gbt']

time: 2.63 ms


In [21]:
model_dict.pop('model', None)

time: 892 µs


In [22]:
model_dict['rfr'].keys()

['Route4_Perf1_rfr',
 'Route2_Perf1_rfr',
 'Route3_Perf1_rfr',
 'Route0_Perf2_rfr',
 'param',
 'Route3_Perf2_rfr',
 'Route1_Perf1_rfr',
 'Route2_Perf2_rfr',
 'Route1_Perf2_rfr',
 'Route1_Perf0_rfr',
 'Route2_Perf0_rfr',
 'Route0_Perf0_rfr',
 'model',
 'Route3_Perf0_rfr',
 'Route4_Perf0_rfr',
 'Route0_Perf1_rfr',
 'Route4_Perf2_rfr']

time: 2.92 ms


In [23]:
model_dict['rfr'].pop('model', None)

RandomForestRegressor_4c22bc3c9c722fb1d3e1

time: 2.84 ms


In [24]:
model_dict['gbt'].pop('model', None)
model_dict['lr'].pop('model', None)
model_dict['dt'].pop('model', None)

DecisionTreeRegressor_4116baa40279c7c31f75

time: 3.57 ms


In [25]:
import json
 
 
json = json.dumps(model_dict)
f = open("model_dict.json","w")
f.write(json)
f.close()



time: 2.97 ms


In [26]:
import pickle
f = open("model_dict.pkl","wb")
pickle.dump(model_dict,f)
f.close()

time: 3.51 ms
